<a href="https://colab.research.google.com/github/bala1802/END_Assignments/blob/main/Session5_Vanilla_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

For Time Series -

* Forecasting - many-to-many or many-to-one

* Classification - many-to-one


For NLP -
* Text Classification: many-to-one

* Text Generation: many-to-many

* Machine Translation: many-to-many

* Named Entity Recognition: many-to-many

* Image Captioning: one-to-many

In [2]:
data = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [3]:
print("Data: ", data.shape, "\n\n", data)

Data:  torch.Size([20]) 

 tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 16., 17., 18., 19., 20.])


* From here, the input data is 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

* Dividing the input data into 4 batches of sequence length = 5


[[1, 2, 3, 4, 5],

[6, 7, 8, 9, 10],

[11, 12, 13, 14, 15],

[16, 17, 18, 19, 20]]

* Batch Size = 4
* Sequence Length = 5
* Input Size = 1 (Since, only one dimension)

**Input:**

torch.rnn has 2 inputs: `input` and `h_0`

`input` size will be of `(sequence_length, batch, input_size)` in our example, `(5, 4, 1)`

`h_0` initial hidden state of the network. size will be of `(num_layers * num_directions, batch, input_size)` in our example, `(num_layers * num_directions, 4, 1)`

**Output:**

torch.rnn has 2 outputs: `output` and `hidden` (`h_n`)

`output` size will be of `(sequence_length, batch, num_directions * hidden_size)` 

`h_n` size will be of `(num_layers * num_directions, batch, hidden_size)`


In [5]:
# Number of features used as input. (Number of columns)
INPUT_SIZE = 1

# Number of previous time stamps taken into account.
SEQ_LENGTH = 5

# Number of features in last hidden state ie. number of output timesteps to predict
HIDDEN_SIZE = 2

# Number of stacked rnn layers.
NUM_LAYERS = 1

# We have total of 20 rows in our input. 
# We divide the input into 4 batches where each batch has only 1
# row. Each row corresponds to a sequence of length 5. 
BATCH_SIZE = 4

**Input:**

torch.rnn has 2 inputs: `input` and `h_0`

`input` size will be of `(sequence_length, batch, input_size)` in our example, `(5, 4, 1)`

`h_0` initial hidden state of the network. size will be of `(num_layers * num_directions, batch, input_size)` in our example, `(1 * num_directions, 4, 1)`

**Output:**

torch.rnn has 2 outputs: `output` and `hidden` (`h_n`)

`output` size will be of `(sequence_length, batch, num_directions * hidden_size)` in our example `(5, 4, num_directions * 2)`

`h_n` size will be of `(num_layers * num_directions, batch, hidden_size)` in our example `(1 * num_directions, 4, 2)`


In [6]:
#initializing the RNN
rnn = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers = 1, batch_first=True)

# input size : (batch, seq_len, input_size)
inputs = data.view(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)

# out shape = (batch, seq_len, num_directions * hidden_size)
# h_n shape  = (num_layers * num_directions, batch, hidden_size)
out, h_n = rnn(inputs)

In [11]:
print('Input: ', inputs.shape, '\n', inputs) #(4,5,1) = (batch_size, seq_len, input_size)
print('\nOutput: ', out.shape, '\n', out) #(4, 5, 2) - (batch_size, seq_len, num_directions * hidden_size) num_directions = 1, hidden_size = 2
print('\nHidden: ', h_n.shape, '\n', h_n) #(1, 4, 2) - (num_layers * num_directions, batch, hidden_size) num_layers = 1, num_directions = 1

Input:  torch.Size([4, 5, 1]) 
 tensor([[[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.]],

        [[ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]],

        [[11.],
         [12.],
         [13.],
         [14.],
         [15.]],

        [[16.],
         [17.],
         [18.],
         [19.],
         [20.]]])

Output:  torch.Size([4, 5, 2]) 
 tensor([[[ 0.3609, -0.6926],
         [ 0.7274, -0.5721],
         [ 0.8379, -0.7582],
         [ 0.9022, -0.7705],
         [ 0.9325, -0.8169]],

        [[ 0.8268, -0.8828],
         [ 0.9639, -0.8497],
         [ 0.9763, -0.8941],
         [ 0.9834, -0.9095],
         [ 0.9881, -0.9253]],

        [[ 0.9624, -0.9582],
         [ 0.9938, -0.9466],
         [ 0.9956, -0.9584],
         [ 0.9968, -0.9658],
         [ 0.9977, -0.9721]],

        [[ 0.9923, -0.9855],
         [ 0.9988, -0.9812],
         [ 0.9991, -0.9849],
         [ 0.9994, -0.9878],
         [ 0.9995, -0.9901]]], grad_fn=<Transpose

In the output above, notice the last row in each batch of `out` is present in `h_n`.

* `out` is the output value at all time-steps of the last RNN layer for each batch.

* `h_n` is the hidden value at the last time-step of all RNN layers for each batch.

**Stacked RNN**

`num_layers = 3`

In [12]:
# Initialize the RNN.
rnn = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers = 3, batch_first=True)

# input size : (batch_size , seq_len, input_size)
inputs = data.view(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)

# out shape = (batch, seq_len, num_directions * hidden_size)
# h_n shape  = (num_layers * num_directions, batch, hidden_size)
out, h_n = rnn(inputs)

In [13]:
print('Input: ', inputs.shape, '\n', inputs) #(4,5,1) = (batch_size, seq_len, input_size)
print('\nOutput: ', out.shape, '\n', out) #(4, 5, 2) - (batch_size, seq_len, num_directions * hidden_size) num_directions = 1, hidden_size = 2
print('\nHidden: ', h_n.shape, '\n', h_n) #(3, 4, 2) - (num_layers * num_directions, batch, hidden_size) num_layers = 3, num_directions = 1

Input:  torch.Size([4, 5, 1]) 
 tensor([[[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.]],

        [[ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]],

        [[11.],
         [12.],
         [13.],
         [14.],
         [15.]],

        [[16.],
         [17.],
         [18.],
         [19.],
         [20.]]])

Output:  torch.Size([4, 5, 2]) 
 tensor([[[0.6081, 0.4663],
         [0.7592, 0.6240],
         [0.7163, 0.6406],
         [0.7470, 0.6262],
         [0.7036, 0.6339]],

        [[0.5675, 0.4616],
         [0.7137, 0.6033],
         [0.6372, 0.6187],
         [0.6968, 0.5945],
         [0.6605, 0.6170]],

        [[0.5442, 0.4587],
         [0.6968, 0.5927],
         [0.6246, 0.6134],
         [0.6896, 0.5898],
         [0.6557, 0.6147]],

        [[0.5416, 0.4584],
         [0.6950, 0.5915],
         [0.6233, 0.6128],
         [0.6888, 0.5893],
         [0.6552, 0.6145]]], grad_fn=<TransposeBackward1>)

Hidden:  torch.Size([3, 4, 

**Bidirectional RNN:**

`bidirectional=True`

In [15]:
rnn = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, batch_first=True, num_layers = 1, bidirectional = True)

# input size : (batch_size , seq_len, input_size)
inputs = data.view(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)

# out shape = (batch, seq_len, num_directions * hidden_size)
# h_n shape  = (num_layers * num_directions, batch, hidden_size)
out, h_n = rnn(inputs)

In [17]:
print('Input: ', inputs.shape, '\n', inputs) #(4,5,1) = (batch_size, seq_len, input_size)
print('\nOutput: ', out.shape, '\n', out) #(4, 5, 4) - (batch_size, seq_len, num_directions * hidden_size) num_directions = 2, hidden_size = 2
print('\nHidden: ', h_n.shape, '\n', h_n) #(2, 4, 2) - (num_layers * num_directions, batch, hidden_size) num_layers = 1, num_directions = 2

Input:  torch.Size([4, 5, 1]) 
 tensor([[[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.]],

        [[ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]],

        [[11.],
         [12.],
         [13.],
         [14.],
         [15.]],

        [[16.],
         [17.],
         [18.],
         [19.],
         [20.]]])

Output:  torch.Size([4, 5, 4]) 
 tensor([[[ 0.0276, -0.4332, -0.2255,  0.8560],
         [-0.1846,  0.1208, -0.5242,  0.8499],
         [ 0.1757,  0.6307, -0.7622,  0.8552],
         [ 0.1578,  0.7798, -0.9021,  0.8696],
         [ 0.2214,  0.9213, -0.9709,  0.9170]],

        [[-0.0211,  0.9810, -0.9857,  0.8987],
         [ 0.3852,  0.9925, -0.9947,  0.9121],
         [ 0.1423,  0.9961, -0.9981,  0.9238],
         [ 0.2815,  0.9990, -0.9993,  0.9345],
         [ 0.1901,  0.9996, -0.9998,  0.9589]],

        [[-0.0698,  0.9999, -0.9999,  0.9505],
         [ 0.3769,  1.0000, -1.0000,  0.9572],
         [ 0.1026,  1.0000, -1.0000,  